# HW1 - part 6

In [1]:
import sys, random,binascii,AES,jupyter_utils,random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display,HTML
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150


This section will focus on the Diffusion principle and examine the extent to which a change in input or internal states affects the end result. The runs in the strain part will be performed on a single input block (16 characters)

In [2]:
plaintext = "From the Pope to"
print("Total Length:",len(plaintext))
display(Markdown(f'\n"**_{plaintext}_**"'))


Total Length: 16



"**_From the Pope to_**"

In [3]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8 ):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def count_flips_to_convert(a, b):

    diff = a ^ b

    # count number of ones in diff
    count = 0
    while diff:
        diff &= (diff - 1)
        count += 1
    return count

def flip_plaintext(plaintext):
    bitflip_p = tobits(plaintext)
    place = random.choice(range(len(bitflip_p)))
    bitflip_p[place] = int(1 - bitflip_p[place])
    bitflip_plaintext = frombits(bitflip_p)
    return bitflip_plaintext,place
bitflip_plaintext,place = flip_plaintext(plaintext)
print("The bit which was chosen to be replaced: ", place)
print("Result:",bitflip_plaintext)

The bit which was chosen to be replaced:  118
Result: From the Pope vo


And we will try to encrypt the message, using an encryption key **"R is a Short key"**
 

In [4]:
key = "R is a Short key"
print(f"'R is a Short key':\n 0x{key.encode().hex()}")

keys = 7*[key]
place = 7*[""]
aes_lib = 7*[""]

aes_lib[0] = AES.AES(int(keys[0].encode().hex(),base=16))
print("0st key wasn't flipped")
for i in range(6):
    keys[i+1],place[i+1] = flip_plaintext(key)
    print(str(i+1) + "st key was flipped at place " + str(place[i+1]))
    aes_lib[i+1] = AES.AES(int(keys[i+1].encode().hex(),base=16))

'R is a Short key':
 0x5220697320612053686f7274206b6579
0st key wasn't flipped
1st key was flipped at place 57
2st key was flipped at place 112
3st key was flipped at place 82
4st key was flipped at place 104
5st key was flipped at place 125
6st key was flipped at place 0


## Question 4:
1. Run the AES algorithm, and save the intermediate results after each round (including the final result)
2. Randomly select a bit in the login message and change it from 0 -> 1 or 1-> 0 accordingly Repeat (1) and calculate how many bits changed their value as a result of the change of the single bit in each round, and the encrypted message (relative to the same situation when no errors were injected)
4. Repeat the experiment 5 times when each time you change one bit in relation to the message in (1) at the end of the runs, present a table

In [5]:
def run_aes_stages(plaintext, key_num):
    spec_enc_text = [""]*10
    for i in range(0, len(plaintext), 16):  
        ciper = aes_lib[key_num].encrypt_by_stage(int(plaintext[i:i+16].encode().hex(),base=16))
        for j,c in enumerate(ciper):
            spec_enc_text[j] += ('{:x}'.format(c)).zfill(32)
    return spec_enc_text
spec_enc_text = run_aes_stages(plaintext, 0)
display(spec_enc_text)

['cb95e6a2a56a602164f8bdc338622f85',
 'cbf42cb3f27bbb6266e1e7abbc87199e',
 '5953b07e2a304dc27be50e6355176310',
 '203d5627e023761e3d86efd4ace38e38',
 'b874b055f6f6fc733e8c7952dbf7ea9c',
 '1dba47880ed16f3c72d966c2526347cc',
 '737bf4f4317604824733b3b13695b2cf',
 'd0d5082d7e23c65c2e2cd9304833f9f6',
 '54d92855bb1e7c8075c0eb813950e85d',
 '793ec12a27911757630e1fd3c4affd06']

In [6]:
print("Displaying output with another key")
bitflip_spec_enc_text = run_aes_stages(plaintext, 1)
display(bitflip_spec_enc_text)

Displaying output with another key


['cb95e6a2a56a606159c5faf938622fc5',
 'ad3c3e86c0aeb3c57f7bb60c0b3034ab',
 'b1c96036d9513b9058edfed32e62b732',
 '55360b2e9e0d601cd9053bee0f2dcd64',
 '90a537c730d51277e1b1a4bbc11744ef',
 'f74f017325ac61aa7345495efff04cae',
 '72dec50a0cf93e2ac04cc9c6b0b1fc38',
 '0b2e3924ccd2fac9721f33125558234f',
 '9f53ee84108d184c2144d57a25e71148',
 '613a5c6dbd7d587b5ac9556ee977165f']

In [7]:
def calulate_diffrence(textlist,flip_textlist):
    diff = []
    for i in range(len(flip_textlist)):
        p =  int(textlist[i],base=16)
        c =  int(flip_textlist[i],base=16)
        diff.append(count_flips_to_convert(p,c))
    return diff
diff = calulate_diffrence(bitflip_spec_enc_text,spec_enc_text)
display(diff)

[20, 62, 58, 60, 66, 67, 70, 69, 67, 65]

In [15]:
table_result = np.zeros((7,len(bitflip_spec_enc_text)))

for i in range(6):
    bfp_stage = run_aes_stages(plaintext, key_num=1+i)
    table_result[i] = calulate_diffrence(bfp_stage,spec_enc_text)


In [16]:
table_result[6] = np.mean(table_result[:6], axis = 0)
display(pd.DataFrame(table_result, columns=[f"Round {col+1}" for col in range(len(bitflip_spec_enc_text))]))

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,Round 9,Round 10
0,20.000000,62.0,58.000000,60.000000,66.000000,67.000000,70.0,69.000000,67.000000,65.000000
1,59.000000,60.0,63.000000,69.000000,60.000000,58.000000,60.0,73.000000,57.000000,63.000000
2,17.000000,53.0,71.000000,68.000000,64.000000,65.000000,60.0,67.000000,56.000000,65.000000
3,64.000000,69.0,74.000000,67.000000,72.000000,65.000000,56.0,65.000000,67.000000,60.000000
4,23.000000,72.0,55.000000,66.000000,71.000000,64.000000,67.0,72.000000,58.000000,64.000000
5,26.000000,62.0,65.000000,65.000000,65.000000,69.000000,50.0,58.000000,69.000000,72.000000
6,34.833333,63.0,64.333333,65.833333,66.333333,64.666667,60.5,67.333333,62.333333,64.833333
